# Vegetation Phenology <img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[s2_l2a](https://explorer.digitalearth.africa/s2_l2a)


## Background
Phenology is the study of plant and animal life cycles in the context of the seasons.
It can be useful in understanding the life cycle trends of crops and how the growing seasons are affected by changes in climate.
For more information, see the [USGS page on deriving phenology from NDVI time-series](https://www.usgs.gov/land-resources/eros/phenology/science/deriving-phenological-metrics-ndvi?qt-science_center_objects=0#qt-science_center_objects)

## Description

This notebook demonstrates how to calculate vegetation phenology statistics using the DE Africa function [xr_phenology](../Scripts/deafrica_temporal_statistics.py).  To detect changes in plant life for Sentinel-2, the script uses either the **Normalized Difference Vegetation Index (NDVI)** or the **Enhanced Vegetation Index (EVI)**, which are common proxies for vegetation growth and health.

The outputs of this notebook can be used to assess differences in agriculture fields over time or space and also allow the assessment of growing states such as planting and harvesting.
This notebook demonstrates the following steps:

1. Load cloud-masked Sentinel 2 data for an area of interest.
2. Calculate the vegetation proxy index (NDVI or EVI).
3. Generate a zonal time series of vegetation health
4. Complete and smooth the vegetation timeseries to remove gaps and noise.
5. Calculate phenology statistics on a simple 1D vegetation time series
6. Calculate per-pixel phenology statistics

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Load key Python packages and supporting functions for the analysis.

In [1]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import xarray as xr
import datetime as dt
import os

sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_bandindices import calculate_indices
from deafrica_plotting import display_map, rgb
from deafrica_classificationtools import HiddenPrints
from deafrica_dask import create_local_dask_cluster
import deafrica_temporal_statistics as ts

/env/lib/python3.6/site-packages/datacube/storage/masking.py:4: DeprecationWarning: datacube.storage.masking has moved to datacube.utils.masking
  category=DeprecationWarning)


ModuleNotFoundError: No module named 'setuptools.version.pkg_resources'; 'setuptools.version' is not a package

### Set up a dask cluster

This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the `dask dashboard`, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.


In [ ]:
create_local_dask_cluster()

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [ ]:
dc = datacube.Datacube(app='Vegetation_phenology')

### Analysis parameters

The following cell sets important parameters for the analysis:

* `veg_proxy`: Band index to use as a proxy for vegetation health e.g. `NDVI` or `EVI`
* `lat`: The central latitude to analyse (e.g. `-10.6996`).
* `lon`: The central longitude to analyse (e.g. `35.2708`).
* `buffer`: The number of square degrees to load around the central latitude and longitude.
For reasonable loading times, set this as `0.1` or lower.
* `time_range`: The year range to analyse (e.g. `('2019-01', '2019-06')`).



In [ ]:
# Set the vegetation proxy to use
veg_proxy = 'NDVI'

# Define area of interest
lat = 22.817
lon = 28.518
lon_buffer = 0.0175
lat_buffer = 0.004

# Set the range of dates for the analysis
time_range = ('2018-01', '2018-05')

In [ ]:
# Combine central lat,lon with buffer to get area of interest
lat_range = (lat-lat_buffer, lat+lat_buffer)
lon_range = (lon-lon_buffer, lon+lon_buffer)

## View the selected location
The next cell will display the selected area on an interactive map.
Feel free to zoom in and out to get a better understanding of the area you'll be analysing.
Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [ ]:
display_map(x=lon_range, y=lat_range)

## Load cloud-masked Sentinel-2 data

The first step is to load Sentinel-2 data for the specified area of interest and time range. 
The `load_ard` function is used here to load data that has been masked for cloud, shadow and quality filters, making it ready for analysis.

In [ ]:
# Create a reusable query
query = {
    'y': lat_range,
    'x': lon_range,
    'time': time_range,
    'measurements': ['red', 'green', 'blue', 'nir'],
    'resolution': (-10, 10),
    'output_crs': 'epsg:6933'
}

# Load available data from Landsat 8
ds = load_ard(
    dc=dc,
    products=['s2_l2a'],
    dask_chunks={'x': 2000, 'y': 2000},
    **query,
)

print(ds)

**Once the load is complete**, we can plot some of the images as as a true-colour image using the `rgb` function.  

In [ ]:
rgb(ds, index=[0,5,10], col_wrap=1, size=3)

## Compute band indices

This study measures the presence of vegetation through either the `normalised difference vegetation index (NDVI)` or the `enhanced vegetation index (EVI)`.
The index that will be used is dictated by the `veg_proxy` parameter that was set in the "Analysis parameters" section.

The normalised difference vegetation index (NDVI) requires the `red` and `nir` (near-infra red) bands. 
The formula is

$$
\begin{aligned}
\text{NDVI} & = \frac{(\text{NIR} - \text{Red})}{(\text{NIR} + \text{Red})} \\
\end{aligned}
$$

The Enhanced Vegetation Index requires the `red`, `nir` and `blue` bands.
The formula is

$$
\begin{aligned}
\text{EVI} & = \frac{2.5 \times (\text{NIR} - \text{Red})}{(\text{NIR} + 6 \times \text{Red} - 7.5 \times \text{Blue} + 1)} \\
\end{aligned}
$$


Both indices are available through the [calculate_indices](../Frequently_used_code/Calculating_band_indices.ipynb) function, imported from [deafrica_bandindices.py](../Scripts/deafrica_bandindices.py).
Here, we use `collection='s2'` since we're working with Sentinel-2 data.

In [ ]:
# Calculate the chosen vegetation proxy index and add it to the loaded data set
ds = calculate_indices(ds, index=veg_proxy, collection='s2')

# Restructure the data to work with interpolate etc
ds = ds.chunk({'time': -1})

# Save the NDVI to the dask cluster
ds[veg_proxy] = ds[veg_proxy].persist()

The vegetation proxy index should now appear as a data variable, along with the loaded measurements, in the `ds` object.

## Plot the vegetation index over time

To get an idea of how the vegetation health changes throughout the year(s), we can plot a zonal time series over the region of interest. First we will do a simple plot of the zonal mean of the data.  


In [ ]:
ds.NDVI.mean(['x', 'y']).plot.line('b-^', figsize=(11,4))
plt.title('Zonal mean of vegetation timeseries');

## Interpolate vegetation time-series

Due to the infrequency of satellite passes or the presence of cloud obscuring the ground, our data may have gaps. `Xarray` has inbuilt methods such as `.interpolate_na()` to complete a time-series. While these functions are great for small datasets, they can become prohibitively slow at scale. DE Africa's custom function `fast_completion` will remove any `NaNs` (Not a Number) very quickly, even over relatively large datasets. 

We will test both methods below.

In [ ]:
#fill missing values using xarray's interpolate_na() method
ds.NDVI.interpolate_na(dim='time', method='linear').mean(['x', 'y']).plot.line('b-^', figsize=(11,4))

plt.title('Interpolated vegetation timeseries');

Alternatively, lets complete the timeseries using the DE Africa function `deafrica_temporal_statistics.fast_complete()`

In [ ]:
# complete timeseries using fast_completion
ndvi = ts.fast_completion(ds.NDVI)

# # plot the result
ndvi.mean(['x','y']).plot.line('b-^', figsize=(11,4));

## Smoothing vegetation time-series

We may also wish to smooth a time series to remove noise. This can be accomplished in a number of ways, two of which we explore here.
1. By calculating a `xarray.rolling mean()` over a specified window-size. 
2. By applying the `deafrica_temporal_statistcs.smooth()` function, which relies on the `scipy.signal.wiener` filter.

Below we will demonstrate this by first calculating a `rolling mean` across every three time steps. The larger the window-size, the smoother the curve will be, but at the cost of more granular detail in the curve.

In [ ]:
#smooth the data
ndvi_rolling = ndvi.mean(['x', 'y']).rolling(time=3, min_periods=1).mean()

#plot the smoothed data
ndvi_rolling.plot.line('b-^', figsize=(11, 4))
plt.title('Rolling mean vegetation time-series');

 Alternatively, lets smooth the timeseries using `deafrica_temporal_statistcs.smooth()`

In [ ]:
#smooth the data
ndvi_smooth = ts.smooth(ndvi)

#plot the smoothed data
ndvi_smooth.mean(['x', 'y']).plot.line('b-^', figsize=(11,4))
plt.title("Wiener smoothed vegetation time-series");

## Calculate phenology statistics using `xr_phenology`

The DE Africa function `xr_phenology` can caluclate a number of land-surface phenology statistics that together describe the characteristics of a plant's lifecycle.  The function can calculate the following statistics on either a zonal timeseries (like the one above), or on a per-pixel basis:

            SOS = DOY of start of season
            POS = DOY of peak of season
            EOS = DOY of end of season
            vSOS = Value at start of season
            vPOS = Value at peak of season
            vEOS = Value at end of season
            Trough = Minimum value of season
            LOS = Length of season (DOY)
            AOS = Amplitude of season (in value units)
            ROG = Rate of greening
            ROS = Rate of senescence

where DOY = day-of-year (Jan 1st = 0, Dec 31st = 365).  By default the function will return all the statistics as an `xarray.Dataset`, to return only a subset of these statistics pass a list of the desired statistics to the function e.g. `stats=['SOS', 'EOS', 'ROG']`.

The `xr_phenology` function also allows for interpolating and/or smoothing the time-series in the same way as we did above, the interpolating/smoothing will occur before the statistics are calculated. 

See the [deafrica_temporal_statistics](../Scripts/deafrica_temporal_statistics.py) script for more information on each of the parameters in `xr_phenology`.


### Zonal phenology statistics

To help us understand what these statistics refer too, lets first pass the simpler zonal (mean of all pixels in the image) time-series to the function and plot the results on the same curves as above.

In [ ]:
#calculate phenology on the zonal mean of the dataset
with HiddenPrints():
    zonal_phen = ts.xr_phenology(
        ds.NDVI.mean(['x', 'y']),
        stats=['SOS', 'POS', 'EOS', 'vSOS', 'vPOS', 'vEOS'],
        method_sos="first",
        method_eos="last",
        complete="linear",
        smoothing="rolling_mean").load()
print(zonal_phen)

Plot the results with our statistcs annotated on the plot

In [ ]:
# Use DOY results to create a datetime object
year = str(ds.time.dt.year.values[0]) + " "
eos_dt = dt.datetime.strptime(year + str(zonal_phen.EOS.values), '%Y %j')
sos_dt = dt.datetime.strptime(year + str(zonal_phen.SOS.values), '%Y %j')
pos_dt = dt.datetime.strptime(year + str(zonal_phen.POS.values), '%Y %j')

#create plot
fig, ax = plt.subplots(figsize=(11, 4))
ax.plot(ndvi_rolling.time, ndvi_rolling, 'b-^')

#add start of season
ax.plot(sos_dt, zonal_phen.vSOS, 'or')
ax.annotate('SOS',
            xy=(sos_dt, zonal_phen.vSOS.values),
            xytext=(-15, 20),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

#add end of season
ax.plot(eos_dt, zonal_phen.vEOS, 'or')
ax.annotate('EOS',
            xy=(eos_dt, zonal_phen.vEOS.values),
            xytext=(0, 20),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

#add peak of season
ax.plot(pos_dt, zonal_phen.vPOS, 'or')
ax.annotate('POS',
            xy=(pos_dt, zonal_phen.vPOS.values),
            xytext=(-10, -25),
            textcoords='offset points',
            arrowprops=dict(arrowstyle='-|>'))

plt.ylabel('NDVI')
plt.title('Phenology statistics');

### Per-pixel phenology statistics

We can now calculate the statistics for every pixel in our time-series and plot the results. In the code below we have changed the `method_eos` and `method_sos` parameters to `'median'`, this will estimate the beginning and end of the growing seasons as the median value of the slopes on the each side of the curve. The code below also demonstrates using the `fast_complete` and `wiener` methods for gap-filling and smoothing.

In [ ]:
#calculate phenology stats on every pixel in dataset
with HiddenPrints():
    phen = ts.xr_phenology(
        ds.NDVI,
        method_sos='median',
        method_eos='median',
        complete='fast_complete',
        smoothing='wiener'
    ).load()

print(phen)

The phenology statistics have been calculated seperately for every pixel in the image.  Let's plot each of them to see the results. 

In [ ]:
# set up figure
fig, ax = plt.subplots(nrows=5,
                       ncols=2,
                       figsize=(16, 13),
                       sharex=True,
                       sharey=True)

# set colorbar size
cbar_size = 0.7

# set aspect ratios
for a in fig.axes:
    a.set_aspect('equal')

# start of season
phen.SOS.plot(ax=ax[0, 0],
              cmap='magma_r',
              vmax=300,
              vmin=0,
              cbar_kwargs=dict(shrink=cbar_size))
ax[0, 0].set_title('Start of Season (DOY)')
phen.vSOS.plot(ax=ax[0, 1],
               cmap='YlGn',
               vmax=0.8,
               cbar_kwargs=dict(shrink=cbar_size))
ax[0, 1].set_title('NDVI at SOS')

# peak of season
phen.POS.plot(ax=ax[1, 0],
              cmap='magma_r',
              vmax=300,
              vmin=0,
              cbar_kwargs=dict(shrink=cbar_size))
ax[1, 0].set_title('Peak of Season (DOY)')
phen.vPOS.plot(ax=ax[1, 1],
               cmap='YlGn',
               vmax=0.8,
               cbar_kwargs=dict(shrink=cbar_size))
ax[1, 1].set_title('NDVI at POS')

# end of season
phen.EOS.plot(ax=ax[2, 0],
              cmap='magma_r',
              vmax=300,
              vmin=0,
              cbar_kwargs=dict(shrink=cbar_size))
ax[2, 0].set_title('End of Season (DOY)')
phen.vEOS.plot(ax=ax[2, 1],
               cmap='YlGn',
               vmax=0.8,
               cbar_kwargs=dict(shrink=cbar_size))
ax[2, 1].set_title('NDVI at EOS')

# Length of Season
phen.LOS.plot(ax=ax[3, 0],
              cmap='magma_r',
              vmax=300,
              vmin=0,
              cbar_kwargs=dict(shrink=cbar_size))
ax[3, 0].set_title('Length of Season (DOY)')

# Amplitude
phen.AOS.plot(ax=ax[3, 1],
              cmap='YlGn',
              vmax=0.8,
              cbar_kwargs=dict(shrink=cbar_size))
ax[3, 1].set_title('Amplitude of Season')

# rate of growth
phen.ROG.plot(ax=ax[4, 0],
              cmap='coolwarm_r',
              vmin=-0.02,
              vmax=0.02,
              cbar_kwargs=dict(shrink=cbar_size))
ax[4, 0].set_title('Rate of Growth')

# rate of Sensescence
phen.ROS.plot(ax=ax[4, 1],
              cmap='coolwarm_r',
              vmin=-0.02,
              vmax=0.02,
              cbar_kwargs=dict(shrink=cbar_size))
ax[4, 1].set_title('Rate of Senescence')

plt.tight_layout();

## Conclusions

In the example above, we can see these four fields are following the same cropping schedule and are therefore likely the same species of crop. We can also see there are intra-field differences in the rates of growth, and in the NDVI values at different times of the season, which may be attributable to differences in soil quality, watering intensity, or other farming practices. 

Phenology statistics are a powerful way to summarise the seasonal cycle of a plant's life.  Per-pixel plots of phenology can help us understand the timing of vegetation growth and sensecence across large areas and across diverse plant species as every pixel is treated as an independent series of observations. This could be important, for example, if we wanted to assess how the growing seasons are shifting as the climate warms.  


## Next steps

When you're done, return to the "Analysis parameters" cell, modify some values (e.g. `time_range`,  or `lat`/`lon`) and rerun the analysis.

For advanced users, `xr_phenology` could be used for generating phenology feature layers in a machine learning classifier (see [Machine Learning with ODC](../Real_world_examples/Machine_learning_with_ODC.ipynb) for example of running ML models with ODC data).  `xr_phenology` can be passed inside of the `custom_func` parameter in the `deafrica_classificationtools.collect_training_data()` function, allowing phenological statistics to be computed during the collection of training data. An example would look like this:

    import sys
    sys.path.append('../Scripts')
    from deafrica_temporal_statistics import xr_phenology
    from deafrica_classificationtools import collect_training_data

    def phenology_stats(da):
        stats = xr_phenology(da, complete='fast_complete')
        return stats

    training = collect_training_data(...,
                                     custom_func=phenology_stats)

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** July 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)